Initiate esipy objects with dev configs and pyswagger cache then demonstrate operability by checking the status of the ccp api server

In [1]:
from select import select
from requests import Session
from config import *
from esipy import EsiApp, EsiClient, EsiSecurity, cache
from esipy.exceptions import APIException
import pandas as pd
import numpy as np
import json, itertools
import sqlalchemy as sa
import datetime as dt

import time
from concurrent.futures import ThreadPoolExecutor

from pprint import PrettyPrinter
pp = PrettyPrinter(width=100, compact=True).pprint
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.expand_frame_repr', False)

f_cache = cache.FileCache(path="./f_cache")
# create the eve app interface
esiapp = EsiApp(cache=f_cache).get_latest_swagger

# init the security object
esisecurity = EsiSecurity(redirect_uri=ESI_CALLBACK,
                          client_id=ESI_CLIENT_ID,
                          secret_key=ESI_SECRET_KEY,
                          headers={'User-Agent': ESI_USER_AGENT})

# init the client
esiclient = EsiClient(security=esisecurity,
                      cache=f_cache,
                      headers={'User-Agent': ESI_USER_AGENT})

# -----------------------------------------------------------------------
# Datasa models
# -----------------------------------------------------------------------
from sqlalchemy.orm import declarative_base, relationship, Session
Base = declarative_base()

class Users(Base):
    # our ID is the character ID from EVE API
    __tablename__ = 'users'
    character_id = sa.Column(sa.BigInteger,
                             primary_key=True,
                             autoincrement=False)
    character_name = sa.Column(sa.String(200))
    # EVE SSO Token stuff
    access_token_expires = sa.Column(sa.DateTime())
    character_owner_hash = sa.Column(sa.String(255))
    refresh_token = sa.Column(sa.String(100))
    access_token = sa.Column(sa.String(4096))
    # Token that associates this user with other toons
    link_token = sa.Column(sa.String(100), nullable=True)

    created_date = sa.Column(sa.DateTime, default=dt.datetime.utcnow())

    def __repr__(self):
        return f'<User {self.character_name}: {self.character_id}>'
    # overshadow the USERMIXIN get_id method with our own
    def get_id(self):
        """ Required for flask-login """
        return self.character_id
    def get_linked_toons(self):
        """ helper function to get all toons linked to this user """
        return session.query(Users).filter(
            Users.link_token == self.link_token).all()

    def get_sso_data(self):
        """ Little "helper" function to get formated data for esipy security"""
        return {
            'access_token':
                self.access_token,
            'refresh_token':
                self.refresh_token,
            'expires_in': (self.access_token_expires -
                           dt.datetime.utcnow()).total_seconds()
        }

    def update_token(self, token_response):
        """ helper function to update token data from SSO response """
        self.access_token = token_response['access_token']
        self.access_token_expires = dt.datetime.fromtimestamp(
            time.time() + token_response['expires_in'],)
        if 'refresh_token' in token_response:
            self.refresh_token = token_response['refresh_token']

    def clear_esi_tokens(self):
        """ helper function to clear token data """
        self.access_token = None
        self.access_token_expires = None
        self.refresh_token = None


class InvTypes(Base):
    __tablename__ = 'invTypes'
    typeID = sa.Column(sa.BigInteger, primary_key=True, autoincrement=False)
    groupID = sa.Column(sa.BigInteger, autoincrement=False)
    typeName = sa.Column(sa.String(999))
    description = sa.Column(sa.String(9000), nullable=True)
    mass = sa.Column(sa.Float)
    volume = sa.Column(sa.Float)
    capacity = sa.Column(sa.Float)
    portionSize = sa.Column(sa.BigInteger)
    raceID = sa.Column(sa.BigInteger, nullable=True)
    basePrice = sa.Column(sa.Float, nullable=True)
    published = sa.Column(sa.Boolean)
    marketGroupID = sa.Column(sa.BigInteger, nullable=True)
    iconID = sa.Column(sa.BigInteger, nullable=True)
    soundID = sa.Column(sa.BigInteger, nullable=True)
    graphicID = sa.Column(sa.BigInteger)

    invVolumes = relationship('InvVolumes',
                                 backref="invVolumes",
                                 uselist=False)

    def __repr__(self):
        return f'<Item {self.typeID}: {self.typeName}>'


class InvVolumes(Base):
    __tablename__ = 'invVolumes'
    typeID = sa.Column(sa.BigInteger,
                       sa.ForeignKey(InvTypes.typeID),
                       primary_key=True,
                       autoincrement=False)
    packVolume = sa.Column(sa.Float)

    def __repr__(self):
        return f'<{self.typeID} Packed Volume: {self.volume}>'

#  server_request_names = [k for k in esiapp.op.keys()] OR https://esi.evetech.net/
ccp_resp = esiclient.request(esiapp.op['get_status']())
print(f'Tranquility Status\nresp_code: {ccp_resp.status}\n{ccp_resp.data}')

engine = sa.create_engine("sqlite:///application/eveDB.sqlite3")
session = Session(engine)
statement = sa.select(Users).where(Users.character_name == 'Baron Dashforth')

# list of first element of each row (i.e. User objects)
baron = session.execute(statement).scalars().first()


def gen_auth_esiclient(user):
    """ we use a toon's information to __init__ a unique esiclient to make requests. 
        ie If I want to have all three of my characters
        wallet information displayed; I need 3 esiclients.

    Args:
        user: db.model / Users() -
        Used to pull sso information to __init__ esipy client
    """

    # init the security object
    security = EsiSecurity(redirect_uri=ESI_CALLBACK,
                           client_id=ESI_CLIENT_ID,
                           secret_key=ESI_SECRET_KEY,
                           headers={'User-Agent': ESI_USER_AGENT})
    security.update_token(user.get_sso_data())
    if security.is_token_expired:
        try:
            user.update_token(security.refresh())
        except (APIException, AttributeError):
            user.clear_esi_tokens()
            session.commit()
            print(f'Error refreshing esi token\'s for {user.character_name}',
                  'danger')
            return False

    # init the client
    genclient = EsiClient(security=security,
                          headers={'User-Agent': ESI_USER_AGENT})
    return genclient


def threaded_user_mutli_request(toon):
    """
    Args:
        toon: Users()
    """
    client = gen_auth_esiclient(toon)
    wallet_op = esiapp.op['get_characters_character_id_wallet'](
        character_id=toon.character_id)
    orders_op = esiapp.op['get_characters_character_id_orders'](
        character_id=toon.character_id, token=client.security.access_token)
    transacts_op = esiapp.op['get_characters_character_id_wallet_transactions'](
        character_id=toon.character_id, token=client.security.access_token)
    request_bundle = [wallet_op, orders_op, transacts_op]
    return client.multi_request(request_bundle)





def nested_responses_to_dict(responses):
    """
    Args:
        responses: list of responses from get_user_eve_info()
    returns:
        {character_name: wallet:balance, 
                        orders: orders_data, 
                        transactions: transactions_data,
        __next}
    """
    account_data = {}
    for element in responses:
        toon_id = element[0][0]._Request__p.get('path')['character_id']
        toon = session.execute(
            sa.select(Users).where(Users.character_id == toon_id)).scalars().first()
        account_data[toon.character_name] = {}
        for req, res in element:
            req_title = res._Response__op._Operation__operationId.replace(
                "get_characters_character_id_", "")
            if res.status != 200:
                print(
                    f'Error getting eve info for {toon.character_name}: {req_title}',
                    'danger')
                continue
            account_data[toon.character_name][req_title] = res.data

    return account_data


def account_analysis(account_data):
    # ============market transactions=====================
    cgf = pd.DataFrame.from_records(
        account_data.get("Chelsea's Grin").get('wallet_transactions'))
    bdf = pd.DataFrame.from_records(
        account_data.get('Baron Dashforth').get('wallet_transactions'))
    df = pd.concat([cgf, bdf])
    df.date = pd.to_datetime(df.date.apply(lambda x: x.v.date().isoformat()))
    df = df[df.date >= (pd.to_datetime("today") - pd.Timedelta(days=23))]
    df['total_transact'] = df.quantity * df.unit_price
    
    ndf = df.groupby(['is_buy', 'type_id'], as_index=False).agg({
        'date': 'count',
        'quantity': 'sum',
        'total_transact': 'sum'
    })
    sells = ndf[ndf.is_buy == False]
    buys = ndf[ndf.is_buy == True]
    tst = pd.merge(buys, sells, how="left",
                   on='type_id').drop(columns=['is_buy_x', 'is_buy_y'])
    all_types = pd.read_sql("SELECT typeID, typeName, volume FROM invTypes",
                            engine,
                            index_col='typeID')
    tst = pd.merge(tst,
                   all_types,
                   how="left",
                   left_on='type_id',
                   right_on='typeID').drop(columns=['volume', 'type_id'])
    tst = tst.fillna(0)
    tst.rename(columns={
        'date_x': 'stock_freq',
        'quantity_x': 'total_stock',
        'total_transact_x': 'total_cost',
        'date_y': 'sell_freq',
        'quantity_y': 'stock_sold',
        'total_transact_y': 'total_revenue',
    },
               inplace=True)
    tst['on_hand'] = tst.total_stock - tst.stock_sold
    tst['asp'] = tst.total_revenue / tst.stock_sold
    tst['abp'] = tst.total_cost / tst.total_stock
    tst['ppi'] = tst.asp - tst.abp
    tst['up'] = tst.ppi * tst.on_hand
    tst['rp'] = tst.ppi * tst.stock_sold

    tst.sort_values('rp', ascending=False)

    adf = pd.DataFrame.from_records(
        account_data.get("Baron Dashforth").get('orders'))
    adf = pd.merge(all_types,
                   adf,
                   how='right',
                   left_on='typeID',
                   right_on='type_id').drop(columns=[
                       'type_id', 'region_id', 'range', 'issued',
                       'is_corporation', 'volume', 'duration', 'order_id'
                   ])
    structures = pd.read_sql("SELECT struc_id, name FROM structureMarkets",
                             engine,
                             index_col='struc_id')
    adf = pd.merge(adf,
                   structures,
                   how='left',
                   left_on='location_id',
                   right_on='struc_id').drop(columns=['location_id'])
    gdf = adf.groupby(['typeName']).agg({
        'price': 'mean',
        'volume_remain': 'sum',
        'volume_total': 'sum'
    })
    adf['ur'] = adf.price * adf.volume_remain
    adf = adf.groupby(['name'], as_index=False).agg({
        'ur': 'sum',
        'typeName': "count"
    }).rename(columns={'typeName': 'order_count'})
    tst = pd.merge(tst, gdf, how='left', on='typeName')
    tst.ppi.fillna(tst.price - tst.abp, inplace=True)
    tst.up.fillna(tst.ppi * tst.on_hand, inplace=True)
    tst.fillna(0, inplace=True)
    account_data.get("Baron Dashforth")['orders'] = adf.to_json(
        orient='records')
    account_data.get("Baron Dashforth")['wallet_transactions'] = tst.to_json(
        orient='records')
    account_data.get("Baron Dashforth")['stats'] = (tst.rp.sum(), tst.up.sum())

Tranquility Status
resp_code: 200
{'players': 15434, 'server_version': '2143960', 'start_time': <pyswagger.primitives._time.Datetime object at 0x0000022C29A922F0>}


In [2]:
# version 1.0
def get_user_eve_info(toon):

    results = []
    with ThreadPoolExecutor(max_workers=10) as pool:
        for result in pool.map(threaded_user_mutli_request,
                               toon.get_linked_toons()):
            results.append(result)
        # reset esi tokens to origin character's tokens
    results = nested_responses_to_dict(results)
    account_analysis(results)
    if esiclient.security.access_token != toon.access_token:
        esiclient.security.update_token(toon.get_sso_data())
    return results

responses = get_user_eve_info(baron)
pp(pd.read_json(responses['Baron Dashforth']['wallet_transactions']))

responses['Baron Dashforth']['stats']

    stock_freq  total_stock   total_cost  sell_freq  stock_sold  total_revenue                                typeName  on_hand         asp         abp        ppi          up          rp  price  volume_remain  volume_total
0            9         1500  40935000.00         63        1249    54030840.00                        Multifrequency M      251    43259.28    27290.00   15969.28  4008289.14 19945630.00  37700            251          1500
1            2           28   3301200.00         23          28     6941200.00                        Warp Scrambler I        0   247900.00   117900.00  130000.00        0.00  3640000.00      0              0             0
2            1           22  14295600.00         10          22    27590100.00                                Miner II        0  1254095.45   649800.00  604295.45        0.00 13294500.00      0              0             0
3            1           10    199000.00          4          10      370000.00                              

(4583648719.359778, 156188098.23076317)

In [3]:
# version 2.0
def get_user_eve_info(toon):

    results = []
    with ThreadPoolExecutor(max_workers=10) as pool:
        for result in pool.map(threaded_user_mutli_request,
                               toon.get_linked_toons()):
            results.append(result)
        # reset esi tokens to origin character's tokens
    results = nested_responses_to_dict(results)
    # account_analysis(results)
    if esiclient.security.access_token != toon.access_token:
        esiclient.security.update_token(toon.get_sso_data())
    return results

account_data = get_user_eve_info(baron)
cgf = pd.DataFrame.from_records(
account_data.get("Chelsea's Grin").get('wallet_transactions'))
bdf = pd.DataFrame.from_records(
    account_data.get('Baron Dashforth').get('wallet_transactions'))
df = pd.concat([cgf, bdf])
df.date = pd.to_datetime(df.date.apply(lambda x: x.v.date().isoformat()))
df = df[df.date >= (
    pd.to_datetime("today") -
    pd.Timedelta(days=pd.to_datetime("today").day - 1,
                 hours=pd.to_datetime("today").hour,
                 minutes=pd.to_datetime("today").minute,
                 seconds=pd.to_datetime("today").second,
                 microseconds=pd.to_datetime("today").microsecond - 1))]
df['total_transact'] = df.quantity * df.unit_price

ndf = df.groupby(['is_buy', 'type_id'], as_index=False).agg({
    'date': ['count', lambda x: x.max() - x.min()],
    'quantity': 'sum',
    'total_transact': 'sum'
})
ndf['avg_price'] = ndf.total_transact / ndf.quantity
sells = pd.DataFrame(ndf[ndf.is_buy == False])
buys = pd.DataFrame(ndf[ndf.is_buy == True])
intermediate_df = pd.merge(buys, sells, how = 'outer', on = 'type_id')#.drop(columns = ['is_buy_x', 'is_buy_y'])
# intermediate_df = pd.concat([buys, sells]).sort_values(('date', '<lambda_0>'), ascending=False)
intermediate_df.drop(columns=['is_buy_x', 'is_buy_y',('date_x', '<lambda_0>')], inplace=True)


# flatten multiindex
intermediate_df.columns = [
    ' & '.join(col).rstrip('_') if col[1] != '' else col[0]
    for col in intermediate_df.columns.values
]

intermediate_df.rename(columns={
    'date_x & count': 'buy_freq',
    'date_y & count': 'sell_freq',
    'date_y & <lambda_0>': 'shelf_life',
    'quantity_x & sum': 'buy_quantity',
    'quantity_y & sum': 'sell_quantity',
    'total_transact_x & sum': 'buy_total_value',
    'total_transact_y & sum': 'sell_total_value',
    'avg_price_x': 'buy_avg_price',
    'avg_price_y': 'sell_avg_price'
},
                       inplace=True)
default_values = {
    'buy_freq': 0,
    'sell_freq': 0,
    'shelf_life': pd.Timedelta(0),
    'buy_quantity': 0,
    'sell_quantity': 0,
    'buy_total_value': 0,
    'sell_total_value': 0,
    'buy_avg_price': 0,
    'sell_avg_price': 0
}
intermediate_df.fillna(default_values, inplace=True)
intermediate_df['profit_per_item'] = intermediate_df.sell_avg_price - intermediate_df.buy_avg_price
intermediate_df[
    'realized_profit'] = intermediate_df.profit_per_item * intermediate_df.sell_quantity

intermediate_df['realized_velocity'] = intermediate_df.sell_quantity / intermediate_df.shelf_life.dt.days
intermediate_df['realized_ppd'] = intermediate_df.realized_profit / intermediate_df.shelf_life.dt.days

# pp((sells.total_transact.sum() - buys.total_transact.sum()))
# pp(intermediate_df[intermediate_df.buy_quantity > 0].buy_total.sum())
pp(f'realized profit: {intermediate_df[intermediate_df.buy_quantity > 0].realized_profit.sum()}')
# pp(intermediate_df[intermediate_df.buy_quantity > 0].realized_profit.sum() /
#    intermediate_df[intermediate_df.buy_quantity > 0].buy_total.sum() * 100)

# ================current orders=================
orders_df = pd.DataFrame.from_records(account_data.get("Baron Dashforth").get('orders'))
orders_df.drop(columns=['duration', 'is_corporation', 'range', 'issued'], inplace=True)
orders_df['remaining_order_value'] = orders_df.price * orders_df.volume_remain
orders_df = orders_df.groupby('type_id', as_index=False).agg({'location_id': 'count', 'volume_remain': 'sum', 'volume_total': 'sum', 'remaining_order_value': 'sum'})
orders_df['sell_avg_price'] = orders_df.remaining_order_value / orders_df.volume_remain

orders_df = pd.merge(orders_df,
                     intermediate_df[['type_id', 'buy_avg_price']],
                     how='inner',
                     on='type_id')
orders_df['current_profit_per_item'] = orders_df.sell_avg_price - orders_df.buy_avg_price
orders_df['unrealized_profit'] = orders_df.current_profit_per_item * orders_df.volume_remain
orders_df['current_roi'] =  orders_df.current_profit_per_item/ orders_df.buy_avg_price
# pp(orders_df.sort_values('unrealized_profit',ascending = False).head(10))
# pp(intermediate_df.head(10))
unrealized_profit = orders_df[orders_df.buy_avg_price >0.0].unrealized_profit.sum()
late_revenue = orders_df[orders_df.buy_avg_price == 0.0].remaining_order_value.sum()
total_revenue = orders_df.remaining_order_value.sum()
approx_late_profit = (unrealized_profit / (total_revenue - late_revenue)) * late_revenue
total_profit = unrealized_profit + approx_late_profit
total_roi = total_profit / total_revenue *100
print(f'unrealized_profit: {unrealized_profit}\nlate_revenue: {late_revenue}\napproximate late profit: {approx_late_profit}\ntotal remaining revenue: {total_revenue}\ntotal pending profit: {total_profit}\ntotal roi: {total_roi}')
types_df = pd.read_sql_query('select typeID, typeName from invTypes', session.connection())
orders_df = pd.merge(orders_df, types_df, how='inner', left_on='type_id', right_on='typeID' ).drop(columns=['typeID']).sort_values('unrealized_profit', ascending=False)
intermediate_df = pd.merge(intermediate_df, types_df, how='inner', left_on='type_id', right_on='typeID' ).drop(columns=['typeID']).sort_values('realized_ppd', ascending=False)
pp(orders_df.head(10))
pp(intermediate_df[intermediate_df.buy_avg_price > 0].head(10))
session.close()


'realized profit: 4583648719.359778'
unrealized_profit: 550651872.025707
late_revenue: 1203440000.0
approximate late profit: 211680237.33950368
total remaining revenue: 4333994355.0
total pending profit: 762332109.3652107
total roi: 17.589596268987542
    type_id  location_id  volume_remain  volume_total  remaining_order_value  sell_avg_price  buy_avg_price  current_profit_per_item  unrealized_profit  current_roi                                    typeName
8     28288            1             56            60          1203440000.00     21490000.00           0.00              21490000.00      1203440000.00          inf                            'Augmented' Ogre
2      6635            1            119           136           773381000.00      6499000.00     5550000.00                949000.00       112931000.00         0.17          Dual Modulated Light Energy Beam I
13    45592            1              5             6           263750000.00     52750000.00    33430000.00              

c:\Users\Mothership\Desktop\HermitKrabacus\hkvenv\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
